In [6]:
import os
drive_folder = 'Omdena LIBRAS SLP'

### Colab Env Setup

#### Clone repo code

In [ ]:
!git clone https://github.com/OmdenaAI/SaoPauloBrazilChapter_BrazilianSignLanguage

#### Mount your Google Drive folder that has the necessary data

In [ ]:
from google.colab import drive
drive.mount(f'/content/drive/')

#### Check the files

In [ ]:
os.listdir(f'drive/MyDrive/{drive_folder}/Data')

In [ ]:
os.listdir(f'drive/MyDrive/{drive_folder}/Data/preprocessed')

In [ ]:
os.listdir(f'drive/MyDrive/{drive_folder}/Data/preprocessed/landmarks')

#### Copy the data to the Colab environment repo

In [11]:
data_version = 'v4'

required_files = [
]

required_folders = [
    f'/landmarks/{data_version}',
]

print(required_files + required_folders)

['/landmarks/v4']


In [ ]:
import shutil
source_preprocessed_dir = f'drive/MyDrive/{drive_folder}/Data/preprocessed'
target_preprocessed_dir = 'SaoPauloBrazilChapter_BrazilianSignLanguage/data/preprocessed'

In [ ]:
for required_file in required_files:
  source = f'{source_preprocessed_dir}/{required_file}'
  target = f'{target_preprocessed_dir}/{required_file}'
  print(f'Copying file {required_file}')
  print(f'\tFrom: {source}')
  print(f'\tTo: {target}')
  shutil.copy(source, target, dirs_exist_ok=True)
  print('Copied')

for required_folder in required_folders:
  source = f'{source_preprocessed_dir}/{required_folder}'
  target = f'{target_preprocessed_dir}/{required_folder}'
  print(f'Copying folder {required_folder}')
  print(f'\tFrom: {source}')
  print(f'\tTo: {target}')
  shutil.copytree(source, target, dirs_exist_ok=True)
  print('Copied')

### Set the current working directory

In [1]:
os.getcwd()

'/home/ben/projects/SaoPauloBrazilChapter_BrazilianSignLanguage/code'

Current working directory should be the root directory of the project

In [2]:
on_colab = True
if on_colab:
  # move down one directory
  os.chdir('SaoPauloBrazilChapter_BrazilianSignLanguage')
else:
  # move up one directory
  os.chdir('..')
  
os.getcwd()

'/home/ben/projects/SaoPauloBrazilChapter_BrazilianSignLanguage'

### Set up the python environment

#### Colab Env

In [ ]:
!pip install -r colab_requirements.txt

#### Local Env
If you are working locally, you should already create & activate a virtual environment, so that this notebook can run inside it.

If you haven't already created the virtual environment, you can do so with the following command:

`uv sync --extra model`

Then activate the virtual environment with the following command:

`source .venv/bin/activate` (On Linux/Mac)\
`venv\Scripts\activate` (On Windows)

#### Set the python path to find our code modules

In [3]:
os.environ['PYTHONPATH'] = os.environ.get('PYTHONPATH', '') + ':' + os.path.join(os.getcwd(), 'code')

In [4]:
print(os.environ['PYTHONPATH'])

:/home/ben/projects/SaoPauloBrazilChapter_BrazilianSignLanguage/code


## Configure the experiment settings

### Imports & Directories

In [1]:
import yaml
from pprint import pprint

Config Directory should be in the cloned repo, so shouldn't need to be changed

In [ ]:
config_dir =  os.path.join(os.getcwd(), 'code','model', 'configs')

The Training Run Directory should be located here, unless you want to save experiment logs elsewhere.
The `drive_folder` is set at the beginning of the notebook.

In [ ]:
run_dir = f"../drive/MyDrive/{drive_folder}/Data/runs"

### Resume Previous Training Run

If you are starting a new training run:
- leave the `resume_training` flag as `False` and skip this section.

If you are resuming a previous training run:
- Set the training flag to `True`, and write the name of the run folder name
- All the previous config settings will be used (loaded from the specified directory), so you can skip the `'Edit Configurations'` section.

In [8]:
# set the resume training flag
resume_training = False
# set the run name
run_name = "12345678_654321_LSTM"

# check the run dir is accessible
check_path = os.path.join(run_dir, run_name)
# check the config file is accessible
if os.path.isdir(check_path):
    print(f'Run dir exists: {check_path}')
else:
    print(f'Run dir does not exist: {check_path}')
    if os.path.isdir(run_dir):
        print(f'Can\'t find run dir in: {run_dir}')
    else:
        print(f'Can\'t find main run dir: {run_dir}')

Confirm the config before and after editing:

In [23]:
path = os.path.join(os.getcwd(), 'code','model', 'configs', 'training', 'training.yaml')
with open(path, "r") as f:
    config = yaml.safe_load(f)
print('--Before edit--')
pprint(config)

config['resume'] = resume_training
config['run_dir'] = os.path.join(run_dir, run_name)

print('\n--After edit--')
pprint(config)

--Before edit--
{'device': 'cuda',
 'k_folds': 5,
 'lr': '1e-4',
 'num_epochs': 1,
 'patience': 50,
 'resume': False,
 'run_dir': None,
 'train_batch_size': 64,
 'type': 'cross_validation',
 'val_batch_size': 256}

--After edit--
{'device': 'cuda',
 'k_folds': 5,
 'lr': '1e-4',
 'num_epochs': 1,
 'patience': 50,
 'resume': False,
 'run_dir': None,
 'train_batch_size': 64,
 'type': 'cross_validation',
 'val_batch_size': 256}


Overwrite with the edited config:

In [ ]:
with open(path, "w") as f:
    yaml.safe_dump(config, f, sort_keys=False)

### Edit Configurations

There are many different configurations that can be set for the training process, model parameters, feature engineering, etc.

The config yaml files are located in the `config` directory and be edited directly.

For convenience, the cells below can be used to edit some of the key configuration settings.

#### Set the key configuration settings

In [ ]:
# train config
model = 'RNN'
features = [
    'positions',
    'angles',
    'differences',
    'distances',
    'metadata',
    ]

# training
device = 'cpu'

# dataset
logs_base = run_dir

# augmentation
p_rotate = 0.5
p_noise = 0.5

#### Edit config files with the above settings

In [ ]:
# train config
path = os.path.join(config_dir, 'train_config.yaml')
with open(path, 'r') as f:
    config = yaml.safe_load(f)
    
config['defaults'][3]['model'] = model.lower()
config['defaults'][6]['features'] = features

pprint(config)
with open(path, 'w') as f:
    yaml.safe_dump(config, f, sort_keys=False)

In [ ]:
# training
path = os.path.join(config_dir, 'training/training.yaml')
with open(path, 'r') as f:
    config = yaml.safe_load(f)
    
config['device'] = device

pprint(config)
with open(path, 'w') as f:
    yaml.safe_dump(config, f, sort_keys=False)

In [ ]:
# dataset
path = os.path.join(config_dir, 'dataset/dataset.yaml')
with open(path, 'r') as f:
    config = yaml.safe_load(f)

config['logs_base'] = logs_base

pprint(config)
with open(path, 'w') as f:
    yaml.safe_dump(config, f, sort_keys=False)

In [ ]:
# augmentation
path = os.path.join(config_dir, 'augmentation/base_augs.yaml')
with open(path, 'r') as f:
    config = yaml.safe_load(f)

config['train']['rotate']['p'] = p_rotate
config['train']['noise']['p'] = p_noise

pprint(config)
with open(path, 'w') as f:
    yaml.safe_dump(config, f, sort_keys=False)

### [Optional] Monitor the training progress live with TensorBoard

In [ ]:
%load_ext tensorboard

The magic command below works on Colab or Jupyter Notebook.\
In VSCode, you can use the command palette to open the TensorBoard extension: `>Python: Launch TensorBoard`

In [ ]:
print(run_dir)

../drive/MyDrive/Omdena LIBRAS SLP/Data/runs


Use the path printed above to see all runs, or add the `run_name` to see a specific run. Using a specific run works better when resuming, as the folder called `run_name` won't exist yet otherwise.

In [ ]:
%tensorboard --logdir "../drive/MyDrive/Omdena LIBRAS SLP/Data/runs"

## Start the training

- The process will run for the `num_epochs` specified in the `training.yaml` file.
- If early stopping is enabled, it will stop when the validation loss stops improving with the `patience` parameter specified.
- The training settings & logs will be saved to the `run_dir` directory.
- You can monitor the training progress live with TensorBoard.
- If the process is interrupted, it can be resumed by setting the `resume_training` flag to `True` and using the same `run_name`.

In [ ]:
!python code/model/trainer.py